In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, concatenate,LSTM,Dropout
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
def ncf_model(arch):
  user_input = Input(shape=(1,), name = 'user_input')
  embed_user = Embedding(input_dim=nb_users, output_dim=64, input_length=1)(user_input)
  embed_user = Flatten()(embed_user)


  hotel_input = Input(shape=(1,), name = 'item_input')
  embed_hotel = Embedding(input_dim=nb_hotels, output_dim=64, input_length=1)(hotel_input)
  embed_hotel = Flatten()(embed_hotel)

  layers0 = concatenate([embed_user,embed_hotel])

  if len(arch)>1:

    hidden = Dense(arch[0],activation='relu')(layers0)

    for j in range(1,len(arch)):
      hidden = Dense(arch[j],activation='relu')(hidden)
  else:
    hidden = Dense(8,activation='relu')(layers0) 

  out = Dense(5)(hidden)
  model = Model(inputs=[user_input, hotel_input], 
                  outputs=out)
  
  model.compile(optimizer='adam',loss='mae',metrics=['mae'])

  return model

In [ ]:
df = pd.DataFrame(data={'alpha':[],'beta':[],	'mean mae':[],	'mae_cleanliness':[],
                        'mae_location':[],'mae_rooms':[],'mae_service':[],'mae_sleep':[],'mae_value':[],	
                        'mean rmse':[], 'rmse_cleanliness':[],'rmse_location':[],'rmse_rooms':[],	
                        'rmse_service':[],'rmse_sleep':[],'rmse_value':[]})
df

,alpha,beta,mean mae,mae_cleanliness,mae_location,mae_rooms,mae_service,mae_sleep,mae_value,mean rmse,rmse_cleanliness,rmse_location,rmse_rooms,rmse_service,rmse_sleep,rmse_value


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.neighbors import DistanceMetric
kf = KFold(n_splits=5,random_state=24,shuffle=True)



dist = DistanceMetric.get_metric('hamming')




value_alpha = [0.7,0.5,0.3]
dic_alpha = {}
for alpha in value_alpha:

        dic_alpha[alpha] = {}
        dic_alpha[alpha]['mae_clean'] = []
        dic_alpha[alpha]['mae_location'] = []
        dic_alpha[alpha]['mae_rooms'] = []
        dic_alpha[alpha]['mae_service'] = []
        dic_alpha[alpha]['mae_value'] = []
        dic_alpha[alpha]['mae_mean'] = []

        dic_alpha[alpha]['rmse_clean'] = []
        dic_alpha[alpha]['rmse_location'] = []
        dic_alpha[alpha]['rmse_rooms'] = []
        dic_alpha[alpha]['rmse_service'] = []
        dic_alpha[alpha]['rmse_value'] = []
        dic_alpha[alpha]['rmse_mean'] = []
k=7
for i in range(1,6):
    pred_Bert_cf = []
    train_data = pd.read_csv('/content/drive/MyDrive/fold_with_sentiment'+str(i)+'_train.csv')
    test_data = pd.read_csv('/content/drive/MyDrive/fold_with_sentiment'+str(i)+'_test.csv')
    train_data.shape , test_data.shape
    nb_users = int(max(train_data.user_id.unique())+1)
    nb_hotels = int(max(train_data.offering_id.unique())+1)
    nb_users, nb_hotels

    ncf = ncf_model([128,64,32,16,8])

    y = train_data[['cleanliness','location','rooms','service','value']].values
    y_test = test_data[['cleanliness','location','rooms','service','value']].values
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    ncf.fit([train_data.user_id.values,train_data.offering_id.values],y,validation_split=0.1,callbacks=[callback],
              
              epochs=20,batch_size=128)
    ncf_pred = ncf.predict([test_data.user_id.values,test_data.offering_id.values])
    cmp=0
    for idx,row in test_data.iterrows():
      cmp+=1
      precedent_data = train_data[train_data['user_id']==row['user_id']]
      precedent_hotels = precedent_data.offering_id.values
      prcd_reviews = precedent_data.text.values
      hotel_id = row['offering_id']
      
      if hotel_id in train_data.offering_id.values:
        precedent_data = train_data[train_data['user_id']==row['user_id']]

        if precedent_data.shape[0] > 0:

          precedent_hotels = precedent_data.offering_id.values
          prcd_reviews = precedent_data.sentiment_score.values
          
        
          prcedent_data_sentiment_review = {}
          for h_id,rev in zip(precedent_hotels,prcd_reviews):
            
            
            prcedent_data_sentiment_review[h_id] = rev
            
          
          user = row['user_id']

          maybe_sim_users = []
          sim = {}
          if cmp%100==0:
            print('rest',test_data.shape[0] - cmp)

          users = train_data[train_data['offering_id']==hotel_id].user_id.unique()
          for user1 in users:
            if (user1 != user):
              

              user1_data = train_data[train_data['user_id']==user1]
              sim_items = set(precedent_hotels).intersection(set(user1_data.offering_id.values))
              if len(sim_items) > 1:
                  
                  maybe_sim_users.append(user1)
                
                  user_rat = []
                  user1_rat = []
                  for item in sim_items:
                    
                    user_rat.append(prcedent_data_sentiment_review[item])

                  for item in sim_items:
                    rev = user1_data[user1_data['offering_id']==item].sentiment_score.values
                    
                    
                    user1_rat.append(rev[0])
                  
                  if (1 - dist.pairwise([user_rat],[user1_rat])[0][0])> 0.5:
                    sim[user1] = 1 - dist.pairwise([user_rat],[user1_rat])[0][0]
          
          if len(sim.keys())>0: #similar users
            rat = [0.0,0.0,0.0,0.0,0.0]
            sum_sim = 0.0
            for user1 in list(sim.keys())[:k]:
              rat += sim[user1] * train_data[(train_data['user_id']==user1) & (train_data['offering_id']==hotel_id ) ][['cleanliness','location','rooms','service','value']].values[0]
              sum_sim+= sim[user1]
            rat = rat / sum_sim
          
          
          else : #no similar users
            rat = [0.0,0.0,0.0,0.0,0.0]
            for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
              rat[i] = np.mean(train_data[train_data['offering_id']==hotel_id][[col]].values)
          
        else: #new_user
            rat = [0.0,0.0,0.0,0.0,0.0]
            for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
              rat[i] = np.mean(train_data[train_data['offering_id']==hotel_id][[col]].values)  

      elif train_data[train_data['user_id']==row['user_id']].shape[0]>0: #new_item
        rat = [0.0,0.0,0.0,0.0,0.0]
        for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
            rat[i] = np.mean(precedent_data[[col]].values)
      else: #new_user , new_item
        #rnd_petubation = random.uniform(-1,1)
        #val = rnd_petubation + 3.0
        rat = [3.0,3.0,3.0,3.0,3.0]
      pred_Bert_cf.append(rat.copy())

    pred_Bert_cf = np.array(pred_Bert_cf)

    
    for alpha in value_alpha:
      pred = alpha*ncf_pred + (1-alpha)*pred_Bert_cf
      pred = np.array(pred)
      dic_alpha[alpha]['mae_clean'].append(mae(pred[:,0],y_test[:,0]))
      dic_alpha[alpha]['mae_location'].append(mae(pred[:,1],y_test[:,1]))
      dic_alpha[alpha]['mae_rooms'].append(mae(pred[:,2],y_test[:,2]))
      dic_alpha[alpha]['mae_service'].append(mae(pred[:,3],y_test[:,3]))
      dic_alpha[alpha]['mae_value'].append(mae(pred[:,4],y_test[:,4]))
      dic_alpha[alpha]['mae_mean'].append(mae(pred,y_test))

      dic_alpha[alpha]['rmse_clean'].append(mse(pred[:,0],y_test[:,0],squared=False))
      dic_alpha[alpha]['rmse_location'].append(mse(pred[:,1],y_test[:,1],squared=False))
      dic_alpha[alpha]['rmse_rooms'].append(mse(pred[:,2],y_test[:,2],squared=False))
      dic_alpha[alpha]['rmse_service'].append(mse(pred[:,3],y_test[:,3],squared=False))
      dic_alpha[alpha]['rmse_value'].append(mse(pred[:,4],y_test[:,4],squared=False))
      dic_alpha[alpha]['rmse_mean'].append(mse(pred,y_test,squared=False))
      

 
      
    


    




/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_distance_metric.py:14: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  category=FutureWarning,


Epoch 1/20
55/55 [==============================] - 4s 8ms/step - loss: 3.4135 - mae: 3.4135 - val_loss: 1.9481 - val_mae: 1.9481
Epoch 2/20
55/55 [==============================] - 0s 5ms/step - loss: 1.4899 - mae: 1.4899 - val_loss: 1.3715 - val_mae: 1.3715
Epoch 3/20
55/55 [==============================] - 0s 5ms/step - loss: 1.0519 - mae: 1.0519 - val_loss: 1.1298 - val_mae: 1.1298
Epoch 4/20
55/55 [==============================] - 0s 5ms/step - loss: 0.7917 - mae: 0.7917 - val_loss: 0.9605 - val_mae: 0.9605
Epoch 5/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6774 - mae: 0.6774 - val_loss: 0.9194 - val_mae: 0.9194
Epoch 6/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6601 - mae: 0.6601 - val_loss: 0.9371 - val_mae: 0.9371
Epoch 7/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6444 - mae: 0.6444 - val_loss: 0.8963 - val_mae: 0.8963
Epoch 8/20
55/55 [==============================] - 0s 5ms/step - loss: 0.6306 - mae: 0.63

In [ ]:
pd.read_csv('/content/drive/MyDrive/fold_with_sentiment'+str(i)+'_train.csv')

,overall,cleanliness,location,rooms,service,value,title,text,username,num_cities,num_helpful_votes,num_reviews,num_type_reviews,id,date_stayed,offering_id,date,via_mobile,user_id,sentiment_score
0,4.0,5.0,5.0,5.0,4.0,3.0,“Its the best of the Andaz Brand in the US....”,stayed us andaz properties one best much bette...,JamesE339,34.0,0,104.0,49.0,147612823,December 2012,1927.0,"December 17, 2012",False,626.0,1
1,4.0,5.0,5.0,5.0,4.0,5.0,“Stunningly Wonderful!”,hotels nyc space challenged would well take no...,romadaro,70.0,5,103.0,85.0,147531970,December 2012,1927.0,"December 16, 2012",False,677.0,1
2,5.0,5.0,4.0,5.0,5.0,3.0,“Unexpectedly good”,campus hotels little unpredictable one star ro...,Grotshops,35.0,0,96.0,51.0,140624089,September 2012,849.0,"September 18, 2012",False,271.0,0
3,4.0,4.0,3.0,4.0,4.0,4.0,"“Pleasant enough, but a bit overrated”",recent ever-so-brief new york visit days night...,pillowsofwanderlust,20.0,2,57.0,49.0,145940048,November 2012,1929.0,"November 22, 2012",False,224.0,1
4,4.0,4.0,4.0,5.0,4.0,4.0,“nice suprise”,booked hotel thinking another hilton property ...,beatricepugzilla,28.0,2,41.0,36.0,127060765,March 2012,849.0,"April 1, 2012",False,583.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7730,5.0,5.0,5.0,4.0,5.0,3.0,"“Great Location, would stay again”",nice hotel decor might everyones taste quite d...,CityGirl69,6.0,0,6.0,6.0,25241032,February 2009,353.0,"February 27, 2009",False,824.0,0
7731,4.0,5.0,5.0,5.0,4.0,4.0,“A nice hotel in a nice location - with staff ...,initial confusion part sent us palomar arlingt...,managementtrainer1,8.0,1,16.0,5.0,21791221,October 2008,353.0,"November 12, 2008",False,376.0,0
7732,5.0,4.0,3.0,5.0,4.0,4.0,“Hotel Palomar gets everything right”,nicely designed hotel quiet clean decent sized...,Mattonholiday,6.0,0,6.0,6.0,14683661,March 2008,353.0,"March 31, 2008",False,64.0,1
7733,3.0,4.0,5.0,4.0,4.0,4.0,“Friendly staff-noisy doors everywhere”,pros\n-friendly staff particularly doormen val...,larklaw,43.0,2,54.0,43.0,11093658,November 2007,353.0,"November 27, 2007",False,470.0,0


In [ ]:
!mkdir '/content/drive/MyDrive/trip_dt_mul'


%cp '/content/drive/MyDrive/Trip_multi_fold_with_sentiment1_train.csv'  '/content/drive/MyDrive/trip_dt_mul/'
%cp '/content/drive/MyDrive/Trip_multi_fold_with_sentiment1_test.csv'  '/content/drive/MyDrive/trip_dt_mul/'

%cp '/content/drive/MyDrive/Trip_multi_fold_with_sentiment2_train.csv'  '/content/drive/MyDrive/trip_dt_mul/'
%cp '/content/drive/MyDrive/Trip_multi_fold_with_sentiment2_test.csv'  '/content/drive/MyDrive/trip_dt_mul/'


%cp '/content/drive/MyDrive/Trip_multi_fold_with_sentiment3_train.csv'  '/content/drive/MyDrive/trip_dt_mul/'
%cp '/content/drive/MyDrive/Trip_multi_fold_with_sentiment3_test.csv'  '/content/drive/MyDrive/trip_dt_mul/'

%cp '/content/drive/MyDrive/Trip_multi_fold_with_sentiment4_train.csv'  '/content/drive/MyDrive/trip_dt_mul/'
%cp '/content/drive/MyDrive/Trip_multi_fold_with_sentiment4_test.csv'  '/content/drive/MyDrive/trip_dt_mul/'

%cp '/content/drive/MyDrive/Trip_multi_fold_with_sentiment5_train.csv'  '/content/drive/MyDrive/trip_dt_mul/'
%cp '/content/drive/MyDrive/Trip_multi_fold_with_sentiment5_test.csv'  '/content/drive/MyDrive/trip_dt_mul/'

mkdir: cannot create directory ‘/content/drive/MyDrive/trip_dt_mul’: File exists


In [ ]:
dic_alpha

{0.3: {'mae_clean': [0.7615778903268179,
   0.7678412040494637,
   0.7435934867137031,
   0.7331988387267072,
   0.7728479189789685],
  'mae_location': [0.6549028874152865,
   0.665609373815777,
   0.6234117435274813,
   0.6531059591326951,
   0.6430278127524098],
  'mae_mean': [0.8185127666057099,
   0.811254536724355,
   0.8096892576485445,
   0.8068898564471587,
   0.8224245612102257],
  'mae_rooms': [0.8552351412716566,
   0.8510805368148716,
   0.8669278221247365,
   0.8580116349392676,
   0.848806848367085],
  'mae_service': [0.8707434171347097,
   0.8522020635840604,
   0.8745178628502328,
   0.8585069581273979,
   0.8912846761837243],
  'mae_value': [0.9501044968800777,
   0.9195395053575967,
   0.9399953730265658,
   0.9316258913097215,
   0.9561555497689357],
  'rmse_clean': [1.0476817905653375,
   1.0519083605904693,
   1.0248612698677573,
   1.00704810957343,
   1.0517561889220148],
  'rmse_location': [0.9053560142330266,
   0.9082854635027591,
   0.8763508610343725,
   0.8

In [ ]:
dic_final = {}
for alpha in value_alpha:
  dic_final[alpha] = {}
  for key in dic_alpha[alpha].keys():
    dic_final[alpha][key] = np.mean(dic_alpha[alpha][key])

In [ ]:
dic_final

{0.3: {'mae_clean': 0.7558118677591321,
  'mae_location': 0.6480115553287299,
  'mae_mean': 0.8137541957271986,
  'mae_rooms': 0.8560123967035235,
  'mae_service': 0.869450995576025,
  'mae_value': 0.9394841632685795,
  'rmse_clean': 1.0366511439038018,
  'rmse_location': 0.8990339737937718,
  'rmse_mean': 1.0872608894099702,
  'rmse_rooms': 1.1173359287798523,
  'rmse_service': 1.175052768070093,
  'rmse_value': 1.2082306325023329},
 0.5: {'mae_clean': 0.7583291975285027,
  'mae_location': 0.6613881868266664,
  'mae_mean': 0.8142887576561396,
  'mae_rooms': 0.8564743283458138,
  'mae_service': 0.863809337598582,
  'mae_value': 0.9314427379811325,
  'rmse_clean': 1.0352769209382622,
  'rmse_location': 0.90522695753534,
  'rmse_mean': 1.0846051520181847,
  'rmse_rooms': 1.11658262236105,
  'rmse_service': 1.1666356237042552,
  'rmse_value': 1.1993036355520332},
 0.7: {'mae_clean': 0.7674407825615626,
  'mae_location': 0.6827675303860815,
  'mae_mean': 0.8223394625247575,
  'mae_rooms': 

In [ ]:
import json
with open("/content/drive/MyDrive/finally_NCF+BERT_cf_with_callback.json", "w") as outfile:
    json.dump(dic_final, outfile)

#Alpha 0.7 beta 0.3

In [ ]:
pred = 0.7*ncf_pred + 0.3*pred_Bert_cf
mae(pred,y_test)

0.8222287391593394

#Alpha 0.5 beta 0.5

In [ ]:
pred = 0.5*ncf_pred + 0.5*pred_Bert_cf
mae(pred,y_test)

0.8178455189825333

#Alpha 0.3 beta 0.7

In [ ]:
pred = 0.3*ncf_pred + 0.7*pred_Bert_cf
mae(pred,y_test)

0.8234194200785353